# Create a dataset with phonemes


In [1]:
%cd ..
%ls

/Users/daniilrobnikov/Developer/TTS/vits-bengali
LICENSE                         preprocess.py
README.md                       requirements.txt
attentions.py                   requirements_py11.txt
commons.py                      resources/
configs/                        test-env.txt
data_utils.py                   test-gpu_monotonic_align.ipynb
filelists/                      test-todo.txt
inference.ipynb                 test_torchaudio.ipynb
losses.py                       text/
mel_processing.py               train.py
models.py                       train_ms.py
modules.py                      transforms.py
monotonic_align.py              utils.py
preprocess/


In [2]:
import pandas as pd
from phonemizer import phonemize

dataset_name = "madasr23"
origin_file_path = ["/Users/daniilrobnikov/Developer/datasets/madasr23dataset/corpus/bn/train/text",
                    "/Users/daniilrobnikov/Developer/datasets/madasr23dataset/corpus/bn/dev/text"]

In [3]:
# Read and combine data from all files
data = pd.concat([pd.read_csv(path, sep="\t", header=None)
                 for path in origin_file_path], ignore_index=True)
print("Number of lines:", len(data))
print(data.head())

# The data looks like this:
# sid_txtid_uttid text
# 16777288_629046_281474981563595 বাঁশের প্রায়

Number of lines: 581236
                                                   0
0  16777288_629046_281474981563595 বাঁশের প্রায় দ...
1  16777288_629072_281474981405386 এক ধরনের পদ্ধত...
2  16777288_629108_281474981563619 গুগলি বা ঝিনুক...
3  16777288_629112_281474981581650 বাগ আরশোলা ইত্...
4  16777288_629121_281474981582236 প্রাকৃতিক সব জ...


In [4]:
# Convert the data to the format of sid txtid uttid text, where each value in own column
data = data[0].str.split("_", n=2, expand=True)

data = pd.concat(
    [data[0], data[1], data[2].str.split(" ", n=1, expand=True)], axis=1)
data.columns = ["spkid", "txtid", "uttid", "text"]
print(data.head())

      spkid   txtid            uttid  \
0  16777288  629046  281474981563595   
1  16777288  629072  281474981405386   
2  16777288  629108  281474981563619   
3  16777288  629112  281474981581650   
4  16777288  629121  281474981582236   

                                                text  
0  বাঁশের প্রায় দশ হাজার প্রজাতি হয় যেমন বাম্বুসা...  
1  এক ধরনের পদ্ধতি যেটা দিয়ে শস্য থেকে খোসা ছাড়ান...  
2         গুগলি বা ঝিনুকের মানে হল ওয়েস্টার আমরা খাই  
3            বাগ আরশোলা ইত্যাদি সব পোকা গুলোর চাষ হয়  
4        প্রাকৃতিক সব জিনিস গুলো দিয়া যখন চাষ করা হয়  


In [5]:
# Convert the text to phonemes
# It may take a while. F.e. 500_000 lines of text ~ 25 minutes
phonemes = phonemize(data["text"], backend="espeak", language="bn",
                     strip=True, preserve_punctuation=True, with_stress=True, njobs=8)

phonemes = pd.DataFrame(phonemes)
phonemes.columns = ["phonemes"]
data = pd.concat([data, phonemes], axis=1)
print(data.head())

      spkid   txtid            uttid  \
0  16777288  629046  281474981563595   
1  16777288  629072  281474981405386   
2  16777288  629108  281474981563619   
3  16777288  629112  281474981581650   
4  16777288  629121  281474981582236   

                                                text  \
0  বাঁশের প্রায় দশ হাজার প্রজাতি হয় যেমন বাম্বুসা...   
1  এক ধরনের পদ্ধতি যেটা দিয়ে শস্য থেকে খোসা ছাড়ান...   
2         গুগলি বা ঝিনুকের মানে হল ওয়েস্টার আমরা খাই   
3            বাগ আরশোলা ইত্যাদি সব পোকা গুলোর চাষ হয়   
4        প্রাকৃতিক সব জিনিস গুলো দিয়া যখন চাষ করা হয়   

                                            phonemes  
0  bˈãʃeɾ pɾˈajo dˈɔʃ hˈaɟaɾ pɾˈoɟatˌi hˈɔjo dʒˈ...  
1  ˈek dʰˈɔɾɔnˌeɾ pˈɔddʰɔtˌi dʒˈeʈa dˈie ʃˈɔssɔ t...  
2  ɡˈuɡɔlˌi bˈa ɟʰˈinukˌeɾ mˈane hˈɔl ˈoːeʃʈˌaɾ ˌ...  
3  bˈaɡ ˈaɾɔʃˌola ˈitædˌi ʃˈɔb pˈoka ɡˈuloɾ tʃˈaʃ...  
4  pɾakɾˈitik ʃˈɔb ɟˈiniʃ ɡˈulo dˈia dʒˈɔkʰɔn tʃˈ...  


### Save the data to .csv file


In [6]:
data.to_csv(f"filelists/{dataset_name}.csv", index=False)